In [22]:
import pandas as pd
import ast

### Race results

In [23]:
df_race_results = pd.read_csv('../data/results_2000-2024.csv')

df_race_results['Results'] = df_race_results['Results'].apply(ast.literal_eval)

rows = []
results_expanded = []
for index, row in df_race_results.iterrows():
    results_list = row['Results']
    for result in results_list:
        rows.append(row.drop('Results'))
        results_expanded.append(result)

expanded_rows_df = pd.DataFrame(rows)
results_normalized_df = pd.json_normalize(results_expanded)

drivers_results_df = pd.concat([expanded_rows_df.reset_index(drop=True), results_normalized_df.reset_index(drop=True)], axis=1)

drivers_results_df['points'] = pd.to_numeric(drivers_results_df['points'], errors='coerce') # convert to integer

## Circuits
drivers_results_df['Circuit'] = drivers_results_df['Circuit'].apply(ast.literal_eval)
# print(type(drivers_results_df['Circuit'].iloc[0]))
circuits_normalized = pd.json_normalize(drivers_results_df['Circuit'])

drivers_results_df = pd.concat([drivers_results_df.drop(columns=['Circuit']), circuits_normalized],axis=1)

drivers_results_df.drop(columns=['url', 'Location.lat',	'Location.long', 'Location.locality', 'Location.country', 'Driver.url', 'Driver.permanentNumber', 'Constructor.url', 'time'], axis=1, inplace=True)

drivers_results_df.rename(columns={'Driver.driverId': 'driverId', 'Driver.code': 'driverCode', 'Driver.givenName': 'driverName', 'Driver.familyName': 'driverSurname', 'Driver.dateOfBirth': 'driverBirth',
'Driver.nationality':'driverNationality', 'Constructor.constructorId': 'constructorId', 'Constructor.name': 'constructorName', 'Constructor.nationality': 'constructorNationality', 'Time.millis': 'timeMillis', 'Time.time': 'time', 'FastestLap.rank': 'fastesLapRank',
'FastestLap.lap': 'fastesLap', 'FastestLap.Time.time': 'fastestLapTime', 'FastestLap.AverageSpeed.speed': 'fastestLapAverageSpeed', 'FastestLap.AverageSpeed.units': 'fastestLapAverageSpeedUnit'},  inplace=True)

drivers_results_df = drivers_results_df.astype({'position': 'int64', 'points': 'int64', 'grid': 'int64', 'laps': 'int64'})

# Fill NA

# Create CSV file
drivers_results_df.to_csv('../data/cleaned_results_2000-2024.csv', index=False)
